In [1]:
from konlpy.tag import Okt
import re

def clean_answer(answer):
    # KoNLPy를 사용한 형태소 분석기 초기화
    okt = Okt()

    # 특수문자 및 공백 제거
    answer = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', answer)

    # 형태소 분석 및 불용어 처리
    words = okt.pos(answer, stem=True)
    words = [word for word, pos in words if pos not in ['Josa', 'Eomi', 'Punctuation', 'Foreign']]

    # 정제된 문장으로 재구성
    cleaned_answer = ' '.join(words)

    return cleaned_answer

# 예제 답변
original_answer = '부동산이 일방적으로 날자를 변경할수는 없습니다나갈사람,들어올사람 ,임대인모두에게 통보해서 날자변경이 협의가 된상태에서 해야합니다.'

# 정제된 답변 얻기
cleaned_answer = clean_answer(original_answer)

# 결과 출력
print(cleaned_answer)


부동산 일방 적 날자 변경 하다 없다 나가다 사람 들어오다 사람 임대 모두 통보 하다 날자 변경 협의 되다 상태 하다 하다


In [2]:
def transform_to_chatbot_response(cleaned_answer):
    # 간단한 패턴 매칭을 통한 챗봇 응답 생성
    if '날자변경' in cleaned_answer:
        response = "날짜 변경은 모든 관계자들과의 협의가 필요합니다."
    elif '일방적' in cleaned_answer:
        response = "부동산 일방적인 날자 변경은 허용되지 않습니다. 협의가 필요합니다."
    else:
        response = "챗봇 응답이 필요한 문장입니다."

    return response

# 예제 정제된 답변
cleaned_answer = clean_answer('부동산이 일방적으로 날자를 변경할수는 없습니다나갈사람,들어올사람 ,임대인모두에게 통보해서 날자변경이 협의가 된상태에서 해야합니다.')

# 챗봇 응답으로 변환
chatbot_response = transform_to_chatbot_response(cleaned_answer)

# 결과 출력
print(chatbot_response)

챗봇 응답이 필요한 문장입니다.


In [3]:
cleaned_answer('날자변경')

TypeError: 'str' object is not callable